In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import linearmodels as lm
import matplotlib
import math
import statsmodels.formula.api as smf
import statsmodels.api as sm

from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import FirstDifferenceOLS
from linearmodels.panel import compare
from matplotlib import pyplot as plt

from pandas.api.types import is_numeric_dtype

pd.set_option('display.max_columns', 500)

%matplotlib inline

C:\Program Files\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# read the main data set
df=pd.read_csv('estimation_file.csv',
                    encoding='utf-8')
df.columns = df.columns.str.strip()

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# exclude if ridership is zero--missing data
df = df[df['UPT_ADJ']>0]

In [4]:
df['BUS_FLAG'] = np.where(df['Mode']=='Bus', 1, 0)
df['RAIL_FLAG'] = np.where(df['Mode']=='Rail', 1, 0)

In [5]:
# keep only 2014 and 2018, and set the index
df = df[(df['Year']==2014) | (df['Year']==2018)]

# set the indices
df['ID'] = df['MNAME'] + '-' + df['Mode']
df=df.set_index(['ID','Year'])

In [6]:
# do not transform categorical fields
category_fields = ['BUS_FLAG', 'RAIL_FLAG', 'CLUSTER_GT', 'CLUSTER_APTA', 'CLUSTER_APTA_HML']

In [7]:
# start by dropping fields already present
drop_fields = []

for col in df.columns: 
    if '_diff' in col: 
        drop_fields.append(col)
    if '_log' in col: 
        drop_fields.append(col)
    if '_scale' in col: 
        drop_fields.append(col)
        
df = df.drop(columns=drop_fields)

In [8]:
# keep only the numeric columns -- the estimation will give an error otherwise
df = df.select_dtypes(include=[np.number])

In [9]:
# calculate the first difference and drop the boundary cases where the CBSA number changes
df = df.sort_index()
for col in df.columns: 
    if not col in category_fields: 
        df[col] = df[col].diff()    
    

In [10]:
# keep only 2018
df = df.reset_index()
df = df[df['Year']==2018]

In [11]:
df.to_csv('change_2014_2018.csv')